# `__init__` vs `__new__`

In [1]:
class Some:
    def __new__(cls, *args, **kwargs):
        print("new", cls, args, kwargs)
        return super().__new__(cls)
    
    def __init__(self, *args, **kwargs):
        print("init", self, args, kwargs)
        

Some("instance", x=0, y=0)

new <class '__main__.Some'> ('instance',) {'x': 0, 'y': 0}
init <__main__.Some object at 0x7f602d5caaf0> ('instance',) {'x': 0, 'y': 0}


💡 The main difference is `__new__` has been called on a class to construct a class instance, when `__init__` has been called exactly on the created instance.

## Subclassing Immutable Types

In [2]:
class UppercaseTuple(tuple):
    def __new__(cls, iterable):
        upper_iterable = (s.upper() for s in iterable)
        return super().__new__(cls, upper_iterable)
    

UppercaseTuple(["hello", "world"])

('HELLO', 'WORLD')

## Singleton

In [3]:
class Settings:
    _instance = None
    
    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__new__(cls, *args, **kwargs)
        return cls._instance
    
    
settings = Settings()
settings2 = Settings()

settings is settings2

True

💡 Defining custom `__new__` method can be also useful for caching data for optimizing some work.

## EncryptedFile

In [11]:
class EncryptedFile:
    _registry = {}
    
    def __init_subclass__(cls, prefix, **kwargs):
        super().__init_subclass__(**kwargs)
        cls._registry[prefix] = cls
        
    def __new__(cls, path: str, key=None):
        prefix, sep, suffix = path.partition(":///")
        if sep:
            file = suffix
        else:
            file = prefix
            prefix = "file"
        
        subclass = cls._registry[prefix]
        obj = object.__new__(subclass)
        obj.file = file
        obj.key = key
        return obj

💡 Pay attention to the `__init_subclass__` method.

In [12]:
class Plaintext(EncryptedFile, prefix="file"):
    def read(self):
        with open(self.file, "r") as f:
            return f.read()

In [13]:
import codecs


class ROT13Text(EncryptedFile, prefix="rot13"):
    def read(self):
        with open(self.file, "r") as f:
            text = f.read()
        return codecs.decode(text, "rot_13")

In [14]:
EncryptedFile("hello.txt").read()

'Hello, World!'

In [15]:
EncryptedFile("rot13:///encrypted.txt", key="dummy").read()

'this text must be kept in secret'